# Community Detection with NetworKit 

In this notebook we will cover some community detection algorithms implemented in the `community` module of NetworKit. Community detection is concerned with identifying groups of nodes which are significantly more densely connected to each other than to the rest of the network. As a first step we import NetworKit:

In [39]:
import networkit as nk
nk.engineering.setNumberOfThreads(1)
nk.engineering.setSeed(0, False)

The `community` module provides a top-level function, [detectCommunities(G, algo=None, inspect=True)](https://networkit.github.io/dev-docs/python_api/community.html?highlight=detect#networkit.community.detectCommunities) to perform community detection of a given graph with a suitable algorithm, and print some statistics about the result. If no algorithm is specified via the `algo` parameter, community detection is performed using the [PLM](https://networkit.github.io/dev-docs/python_api/community.html?highlight=plm#networkit.community.PLM) algorithm.

This function can be used as follows:

In [2]:
# Read graph
G = nk.readGraph("/data/yliumh/AutoAtClusterDatasets/networkit/uk-2007-05@100000-edgelist.txt", nk.Format.EdgeListTabZero)
# G = nk.readGraph("/data/yliumh/AutoAtClusterDatasets/snap/com-orkut.ungraph.txt", nk.Format.SNAP)
# import networkx as nx
# from sklearn.metrics import normalized_mutual_info_score as NMI, adjusted_rand_score as ARI
# import scipy.sparse as sp
# import numpy as np
# # G = nx.karate_club_graph()
# # labels = [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# nc, r = 10, 1000 # ring of cliques: nc=#nodes in each clique; r=#cliques
# data = np.load(f"/home/yliumh/github/AutoAtCluster/baselines/Louvain/dataset/ring_of_cliques_{nc}_{r}.npz")
# adj_data, adj_row, adj_col = data["data"], data["row"], data["col"]
# adj = sp.coo_matrix((adj_data, (adj_row, adj_col)))
# G = nx.from_scipy_sparse_array(adj)
# G = nk.nxadapter.nx2nk(G)
# print("===")
# communities = nk.community.detectCommunities(G)
# preds = communities.getVector()
# print(NMI(labels, preds), ARI(labels, preds))

In [3]:
G.addEdges

<bound method Graph.addEdges of <networkit.graph.Graph object at 0x7fcbcaec5c50>>

In [4]:
G.addWeightedEdges

<bound method Graph.addWeightedEdges of <networkit.graph.Graph object at 0x7fcbcaec5c50>>

In [3]:
G.addWeightedEdgesOfNode

<bound method Graph.addWeightedEdgesOfNode of <networkit.graph.Graph object at 0x7f3b76b69930>>

In [15]:
print(f"#Nodes= {G.numberOfNodes()}, #Edges= {G.numberOfEdges()}, #Weights= {G.totalEdgeWeight()}")

#Nodes= 100000, #Edges= 2795245, #Weights= 2795245.0


In [9]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
def load_knn_graph(dataset, k, seed):
    try:
        knn_graph_path = f"/home/yliumh/github/AutoAtCluster/baselines/KNN/outputs/knn_adj_{dataset}_{seed}_{k}.npz"
        data = np.load(knn_graph_path)
        adj_data, adj_row, adj_col = data["data"], data["row"], data["col"]
        knn_adj = sp.coo_matrix((adj_data, (adj_row, adj_col)))
        graph = nx.from_scipy_sparse_array(knn_adj)
        return graph
    except Exception as e:
        print(e)
graph = load_knn_graph("amazon-photo", 100, 0)
print(graph)
G = nk.nxadapter.nx2nk(graph, weightAttr="weight")

Graph with 7650 nodes and 8785241 edges


In [8]:
nk.community.Modularity().getQuality(communities, G)

0.841363198808599

In [40]:
nk.components.BiconnectedComponents

networkit.components.BiconnectedComponents

In [30]:
nk.graphtools.randomNeighbor(G,0)

1520

In [13]:
induced_subgraph = nk.graphtools.subgraphAndNeighborsFromNodes

In [27]:
G = nk.Graph(5, weighted=True, directed=False)

In [28]:
G.addEdge(1,1,2)
G.addEdge(1,2,3)
G.addEdge(2,2,4)
G.addEdge(2,3,5)

True

In [29]:
G.numberOfEdges()

4

In [37]:
H1 = induced_subgraph(G, [1,2], False, False)

In [38]:
H1.numberOfEdges()

3

In [32]:
G.weight(1,1)

2.0

In [33]:
G.weight(1,2)

3.0

In [34]:
G.weight(2,2)

4.0

In [31]:
import copy

In [32]:
G3 = copy.deepcopy(G)

In [33]:
G.removeEdge(0, 10000)

In [34]:
G.hasEdge(0,10000)

False

In [35]:
G3.hasEdge(0,10000)

True

In [41]:
import torch
import numpy as np

In [37]:
k = 2
data = torch.randn(3,5).to("cuda:2")

In [38]:
data

tensor([[-0.4070,  0.6396,  0.6234,  1.5170, -1.1509],
        [ 2.1212,  1.2742, -0.6445,  1.2006, -0.4938],
        [-0.8980,  1.3964,  0.0503, -0.0824, -0.4227]], device='cuda:2')

In [39]:
values, indices = data.topk(k, dim=-1, sorted=False)

In [42]:
sub_row = np.repeat(np.arange(0,3), k).tolist()
sub_col = indices.cpu().flatten().tolist()

In [43]:
data[sub_row, sub_col] = -999

In [44]:
data

tensor([[-4.0703e-01, -9.9900e+02,  6.2337e-01, -9.9900e+02, -1.1509e+00],
        [-9.9900e+02, -9.9900e+02, -6.4450e-01,  1.2006e+00, -4.9384e-01],
        [-8.9799e-01, -9.9900e+02, -9.9900e+02, -8.2387e-02, -4.2266e-01]],
       device='cuda:2')

In [45]:
data.topk(k, dim=-1, sorted=False)

torch.return_types.topk(
values=tensor([[ 0.6234, -0.4070],
        [ 1.2006, -0.4938],
        [-0.0824, -0.4227]], device='cuda:2'),
indices=tensor([[2, 0],
        [3, 4],
        [3, 4]], device='cuda:2'))

In [4]:
import numpy as np

In [5]:
aa = np.random.rand(20)

In [6]:
aa.size

20

In [7]:
aa.view(dtype=np.double).size

20

The following sections cover two popular community detection algorithms, `PLM` and `PLP`, and will illustrate how to use them.

## PLM

NetworKit provides a parallel implementation of the well-known Louvain method, which can be found in the [PLM](https://networkit.github.io/dev-docs/python_api/community.html?highlight=plm#networkit.community.PLM) class. It yields a high-quality solution at reasonably fast running times. The constructor `PLM(Graph, refine=False, gamma=0.1, par='balance', maxIter=32, turbo=True, recurse=True)` expects a [networkit.Graph](https://networkit.github.io/dev-docs/python_api/networkit.html?highlight=graph#networkit.Graph) as a mandatory parameter. If the parameter `refine` is set to true, the algorithm performs a second move phase to refine the communities. The parameter `gamma` defines the multi-resolution modularity parameter. The string `par` defines the openmp parallelization strategy. `maxIter` is the maximum number of iterations for move phase. When `turbo` is set to true, the algorithm is faster but uses O(n) additional memory per thread. Set `recurse`to true in order to use recursive coarsening. Refer to [this]( http://journals.aps.org/pre/abstract/10.1103/PhysRevE.89.049902) for more details on recursive coarsening.

In the example below we run PLM with `refine` set to true while leaving the rest of the parameters to their default values."

In [3]:
# Choose and initialize algorithm 
algo = nk.community.PLM(G, True, turbo=True, nm="ps", maxIter=32)
plmCommunities = nk.community.detectCommunities(G, algo=algo)
cnt = algo.getCount()
print(algo.getCount(), sum(cnt))
print(algo.getTiming())
print(algo.getIter())
preds = plmCommunities.getVector()
# print(NMI(labels, preds), ARI(labels, preds))

Communities detected in 0.38636 [s]
solution properties:
-------------------  --------------
# communities            63
min community size       21
max community size    18166
avg. community size    1587.3
imbalance                11.4395
edge cut             263321
edge cut (portion)        0.0942032
modularity                0.838026
-------------------  --------------
[953535, 2325, 142, 63, 71, 930, 0] 957066
{b'coarsen': [11, 0, 0], b'move': [364, 0, 0, 0], b'refine': [0, 0, 0]}
[33, 1]


In [5]:
algo.stopNow() is True

False

In [7]:
1 == True

True

In [8]:
0 == True

False

In [5]:
algo.addKNNGraph

<bound method PLM.addKNNGraph of <networkit.community.PLM object at 0x7ff887b47eb0>>

The output of the `detectCommunities` function is a partition of the nodes of the graph. It is represented by the [Partition](https://networkit.github.io/dev-docs/python_api/networkit.html?highlight=partition#networkit.Partition) data structure, which provides several methods for inspecting and manipulating a partition of a set of elements.

In [4]:
print("{0} elements assigned to {1} subsets".format(plmCommunities.numberOfElements(),
                                                    plmCommunities.numberOfSubsets()))

34 elements assigned to 4 subsets


In [5]:
print("the biggest subset has size {0}".format(max(plmCommunities.subsetSizes())))

the biggest subset has size 12


The contents of a partition object can be written to file in a simple format, in which the `i`-th line contains an integer representing the subset id of node `i`.

In [ ]:
nk.community.writeCommunities(plmCommunities, "output/communtiesPLM.partition")

In [10]:
coarsen_algo = nk.coarsening.ParallelPartitionCoarsening(G, plmCommunities, parallel=True)

In [11]:
coarsen_algo.run()

In [12]:
cg = coarsen_algo.getCoarseGraph()

In [14]:
cg.numberOfNodes()

63

In [15]:
coarsen_to_fine = coarsen_algo.getCoarseToFineNodeMapping()

In [16]:
coarsen_to_fine

{0: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
 

In [17]:
algo

In [18]:
algo.prolong

<cyfunction PLM.prolong at 0x7fdd6bd51cb0>

In [20]:
algo.coarsen(G, plmCommunities)

(<networkit.graph.Graph at 0x7fdd66eb3870>,
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [16]:
# Choose and initialize algorithm 
algo = nk.community.ParallelLeiden(G, iterations=32)
plmCommunities = nk.community.detectCommunities(G, algo=algo)

Communities detected in 0.57400 [s]
solution properties:
-------------------  -------------
# communities           391
min community size        2
max community size    11443
avg. community size     255.754
imbalance                44.6992
edge cut             405444
edge cut (portion)        0.145048
modularity                0.804907
-------------------  -------------


## PLP 

The Label Propagation algorithm is an algorithm for finding communities in a graph. NetworKit provides a parallel implementation, [PLP(G, updateThreshold=none, maxIterations=none)](https://networkit.github.io/dev-docs/python_api/community.html?highlight=plp#networkit.community.PLP). The constructor expects a [networkit.Graph](https://networkit.github.io/dev-docs/python_api/networkit.html?highlight=graph#networkit.Graph) as a mandatory parameter. The parameter `updateThreshold` dictates the number of nodes that have to be changed in each iteration so that a new iteration starts, and `maxIterations` is the maximum number of iterations. `none` is NetworKit constant set to the maximum value of a 64-bit integer.

In [ ]:
# Read graph
G = nk.readGraph("../input/jazz.graph", nk.Format.METIS)

In [ ]:
# Choose and initialize algorithm 
plpCommunities = nk.community.detectCommunities(G, algo=nk.community.PLP(G))

In [ ]:
print("{0} elements assigned to {1} subsets".format(plpCommunities.numberOfElements(),
                                                    plpCommunities.numberOfSubsets()))

In [ ]:
print("the biggest subset has size {0}".format(max(plpCommunities.subsetSizes())))

In [ ]:
nk.community.writeCommunities(plpCommunities, "output/communtiesPLP.partition")